In [2]:
# extracting all functional requirements into df dataframe
# extracting USR1 doc into df_req dataframe

import pandas as pd
import docx
import pandasql as ps
import os
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pandasql import sqldf

 
fileName= Path('E:/MS/Thesis/Ilias/internal/URStest.docx').stem

DatabaseDF = pd.read_excel('E:/MS/Thesis/data set/Database.xlsx')

#df.head(5)


Text = docx.Document('E:/MS/Thesis/Ilias/internal/URStest.docx')
#data={}

for table in Text.tables:
    for row in table.rows:
        for cell in row.cells:
            #print (cell.text)
            #print("-------------------------")
            
            if "LFA" in cell.text:
                #print (cell.text)
                data = [[cell.text for cell in row.cells] for row in table.rows]
            #data = cell.text
#df = pd.DataFrame(data)  
#data = [[cell.text for cell in row.cells] for row in table.rows]
            
#df_req = pd.DataFrame(cell.text)
df_req = pd.DataFrame(data) 
headers =  ["Sequence", "URStest"]
df_req.columns = headers

#writing USR1 requirements dataframe to excel

with pd.ExcelWriter(r'E:\MS\Thesis\data set\extracted req.xlsx') as writer:
    df_req.to_excel(writer,sheet_name='functional req', index=False)
 
      
matched_req = []
soft_com = []

for ind in df_req.index:
    USR = df_req['URStest'][ind]
#     print("USR: ", USR)
#     print("-----------------------------")
    for ind1 in DatabaseDF.index:
        Specification_database = DatabaseDF['database_requirements'][ind1]
#         print("database:",Specification_database )
#         print("-----------------------------")
        matched_token=fuzz.token_sort_ratio(USR,Specification_database)
        if matched_token> 80:
            matched_req.append({"user_id": ind,"user": USR,"requirement": Specification_database,"accuracy": matched_token, "Keyword": DatabaseDF["Keyword"][ind1],
                                "sft_com": DatabaseDF["Software complexity"][ind1]})
            #print(matched_req)    
# masterDf = pd.DataFrame(matched_req)
# masterDf.head()

fuzzy = pd.DataFrame(matched_req)
fuzzy.head()


with pd.ExcelWriter(r'E:\MS\Thesis\data set\fuzzy.xlsx') as writer:
    fuzzy.to_excel(writer,sheet_name='fuzzy', index=False)
    
output = sqldf("select Keyword, round(avg(sft_com),3) as software_complexity from fuzzy group by keyword")

output1 = output.T

# with pd.ExcelWriter(r'F:\MS\Thesis\data set\Dataset.xlsx') as writer:
#     output1.to_excel(writer,sheet_name='dataset' , index=False)
    


In [3]:
print(output1)

                           0          1           2     3              4
Keyword              Battery  Collision  Headlights  Horn  Remotecontrol
software_complexity     66.0      102.0     102.667  58.0          131.0


In [3]:
pip install fuzzywuzzy[speedup]

     ------------------------------------ 101.3/101.3 kB 264.8 kB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
